# REINFORCE Algorithm

### 1. Muestrar {$\tau^i$} de $\pi_{\theta}(a_t|s_t)$ - Correr M trayectorias usando la policy
### 2. Estimar el retorno: $$ R(\tau_i)  \approx \sum_{t=0}^{T}R(s_t^i, a_t^i)$$
### 3. Entrenar un modelo: $$ \nabla_{\theta} J_{\theta} \approx \frac{1}{M} \sum_{i=1}^{M}  R(\tau_i)   \sum_{t=0}^T \nabla_{\theta} log \pi_{\theta}(a_t^i|s_t^i)$$

# Implementación en Keras

### Suponiendo que solo corremos un episodio por iteración

La loss queda (Le ponemos un menos adelante para que tengamos que minimizar):

$$ \huge J_{\theta} =  - R \sum_{t=0}^T log \pi_{\theta}(a_t|s_t)$$
$$ \huge J_{\theta} =  - \sum_{t=0}^T log \pi_{\theta}(a_t|s_t) R $$
$$ \huge J_{\theta} =  \sum_{t=0}^T log \frac{1}{\pi_{\theta}(a_t|s_t)} R $$

Recordando la Entropía cruzada:

$$ \huge H(y_{true}, y_{pred}) = \sum_{i} y_{true_i} log (\frac{1}{y_{pred_i}}) $$

Ejemplo: 

- Sumpongamos que tenemos 3 acciones posibles y la red neuronal predijo $y_{pred}$ = [0.2, 0.3, 0.5]
- Se muestreó la salida y se eligión la acción 2, es decir la acción con probabilidad 0.3
- La $y_{true}$ será [0, 1, 0]

$$ \huge H = 0 log (\frac{1}{0.2}) + 1 log (\frac{1}{0.3}) + 0 log (\frac{1}{0.5}) $$

- Si redefinimos la $y_{true}$ como $y_{true}$ = $y_{true} R$
- La $y_{true}$ queda [0, R, 0]

$$ \huge H = 0 log (\frac{1}{0.2}) + R log (\frac{1}{0.3}) + 0 log (\frac{1}{0.5}) = R log (\frac{1}{\pi_{\theta}(a_t|s_t)}) $$

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# conda install -c conda-forge tensorboardx

In [7]:
from REINFORCE_helper import BaseAgent, format_as_pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
import keras.backend as K
import numpy as np

In [8]:
class ReinforceAgent(BaseAgent):
    # def __init__(self):
    def get_policy_model(self, lr=0.001, hidden_layer_neurons = 128, input_shape=[4], output_shape=2):
        ## Defino métrica - loss normalizada (sin el retorno multiplicando)
        def loss_metric(y_true, y_pred):
            y_true_norm = K.sign(y_true)
            return K.categorical_crossentropy(y_true_norm, y_pred)
        model = Sequential()
        model.add(Dense(hidden_layer_neurons, input_shape=input_shape, activation='relu'))
        model.add(Dense(output_shape, activation='softmax'))

        model.compile(Adam(lr), loss=['categorical_crossentropy'], metrics=[loss_metric])
        return model
    
    def get_action(self, eval=False):
        p = self.model.predict([self.observation.reshape(1, self.nS)])
        if eval is False:
            action = np.random.choice(self.nA, p=p[0]) #np.nan_to_num(p[0])
        else:
            action = np.argmax(p[0])
        action_one_hot = np.zeros(self.nA)
        action_one_hot[action] = 1
        return action, action_one_hot, p
    
    def get_entropy(self, preds, epsilon=1e-12):
        entropy = np.mean(-np.sum(np.log(preds+epsilon)*preds, axis=1)/np.log(self.nA))
        return entropy
    
    def get_discounted_rewards(self, r):
        # Por si es una lista
        r = np.array(r, dtype=float)
        """Take 1D float array of rewards and compute discounted reward """
        discounted_r = np.zeros_like(r)
        running_add = 0
        for t in reversed(range(0, r.size)):
            running_add = running_add * self.gamma + r[t]
            discounted_r[t] = running_add
        return discounted_r 

In [9]:
reinforce_agent = ReinforceAgent('CartPole-v1', n_experience_episodes=3)

In [10]:
reinforce_agent.reset_env()
action, action_one_hot, p = reinforce_agent.get_action()
print('Action:', action)
print('action_one_hot:', action_one_hot)
print('Policy prob dist:', p)

Action: 1
action_one_hot: [0. 1.]
Policy prob dist: [[0.49976465 0.5002353 ]]


In [11]:
reinforce_agent.reset_env()
obs, actions, preds, disc_sum_rews, rewards, ep_returns, ep_len, last_obs, time_step = reinforce_agent.get_experience_episodes(return_ts=True)

In [12]:
format_as_pandas(time_step, obs, preds, actions, rewards, disc_sum_rews, ep_returns, decimals = 3)

,step,observation,policy_distribution,sampled_action,rewards,discounted_sum_rewards,episode_return
0,1,"[-0.042, 0.048, 0.006, 0.016]","[0.501, 0.498]","[1, 0]",1.0,9.955,9.955
1,2,"[-0.041, -0.146, 0.006, 0.311]","[0.503, 0.496]","[1, 0]",1.0,8.964,9.955
2,3,"[-0.044, -0.341, 0.012, 0.606]","[0.505, 0.494]","[1, 0]",1.0,7.972,9.955
3,4,"[-0.051, -0.537, 0.025, 0.903]","[0.507, 0.492]","[1, 0]",1.0,6.979,9.955
4,5,"[-0.062, -0.732, 0.043, 1.203]","[0.509, 0.49]","[0, 1]",1.0,5.985,9.955
...,...,...,...,...,...,...,...
65,16,"[0.002, 0.553, -0.128, -1.149]","[0.552, 0.447]","[1, 0]",1.0,4.990,19.811
66,17,"[0.013, 0.36, -0.151, -0.899]","[0.539, 0.46]","[1, 0]",1.0,3.994,19.811
67,18,"[0.02, 0.167, -0.169, -0.658]","[0.526, 0.473]","[0, 1]",1.0,2.997,19.811
68,19,"[0.023, 0.364, -0.182, -0.999]","[0.543, 0.456]","[0, 1]",1.0,1.999,19.811


In [13]:
reinforce_agent.get_entropy(preds)

0.99831235

# Algoritmo REINFORCE

In [16]:
from REINFORCE_helper import RunningVariance
from time import time

# Pruebas interesantes:
- n_experience_episodes=1, epochs=1, lr=0.001
- n_experience_episodes=5, epochs=1, lr=0.001
- n_experience_episodes=50, epochs=1, lr=0.001
- n_experience_episodes=50, epochs=20, lr=0.001
- n_experience_episodes=50, epochs=50, lr=0.001
- n_experience_episodes=50, epochs=50, lr=0.01

In [21]:
running_variance = RunningVariance()
running_variance.get_mean()

In [27]:
running_variance.get_mean()

63.5107017617322

In [ ]:
reinforce_agent = ReinforceAgent('CartPole-v1', n_experience_episodes=50, EPISODES=2000, 
                                 epochs=1, lr=0.004, gamma=0.999, batch_size=512)
# reinforce_agent = ReinforceAgent('LunarLander-v2', n_experience_episodes=1, EPISODES=2000, epochs=1, lr=0.001)

running_variance = RunningVariance()
initial_time = time()


while reinforce_agent.episode < reinforce_agent.EPISODES:
    obs, actions, preds, disc_sum_rews, rewards, ep_returns, ep_len, last_obs = reinforce_agent.get_experience_episodes()
    for dr in ep_returns:
        running_variance.add(dr)
        
    pseudolabels = actions*ep_returns.reshape(-1, 1)

    history = reinforce_agent.model.fit(obs, pseudolabels, verbose=0, epochs=reinforce_agent.epochs, batch_size=128)
    
    reinforce_agent.log_data(reinforce_agent.episode, 
                      history.history['loss'][0], 
                      np.mean(ep_len), 
                      reinforce_agent.get_entropy(preds), 
                      running_variance.get_variance(), 
                      history.history['loss_metric'][0], 
                      time() - initial_time, np.mean(ep_returns[-1]),
                      rv_normalized = running_variance.get_variance()/running_variance.get_mean()**2
                    )

correr en linea de comando: tensorboard --logdir logs/
Episode: 51
Model on episode 52 improved from -inf to 9.95511979025179. Saved!
Episode: 155
Model on episode 156 improved from 9.95511979025179 to 9.95511979025179. Saved!
Episode: 259
Model on episode 260 improved from 9.95511979025179 to 16.86467762617564. Saved!
Episode: 311
Model on episode 312 improved from 16.86467762617564 to 33.444937900916464. Saved!
Episode: 415
Model on episode 416 improved from 33.444937900916464 to 84.2794427501054. Saved!
Episode: 571